In [206]:
import pandas as pd
import dask as dd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.formula.api as smf
import statistics

from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy import stats

In [222]:
def basePrep(df):
	df['rndf'] = df['rndf'].map({"Yes": 1, "No": 0})
	df['pdynamic'] = df['pdynamic'].map({'new_entrant': 1, 'incumbent': 0})	
	
	cols = ['rndf', 'ddist_bin','ddist_int', 'ddist_abs', 'avg_emp_pp', 'public_count', 'pdynamic', "SNATION_PARTAL".lower()]
	df = df[cols].dropna()
	
	return df

## Try all potential variables

In [208]:
df = pd.read_csv('./data/JV_data_dist.csv')

In [209]:
df = basePrep(df)

In [210]:
cols = ['rndf', 'ddist_bin','ddist_int', 'ddist_abs', 'avg_emp_pp', 'public_count', "SNATION_PARTAL".lower()]
X = (df[cols]).astype(float)
y = df['pdynamic'].astype(float)
X_constant = sm.add_constant(X, prepend=False)

In [211]:
logit_model = GLM(y, X_constant, family=families.Binomial())
logit_results = logit_model.fit()
print(logit_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  729
Model:                            GLM   Df Residuals:                      722
Model Family:                Binomial   Df Model:                            6
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -449.53
Date:                Tue, 14 Jun 2022   Deviance:                       899.06
Time:                        14:41:13   Pearson chi2:                     751.
No. Iterations:                     4   Pseudo R-squ. (CS):            0.07881
Covariance Type:            nonrobust                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
rndf             0.4753      0.296      1.607   

## Filter only R&D alliances

In [212]:
df2 = pd.read_csv('./data/JV_data_dist.csv')

In [213]:
df2 = df2[df2['rndf'] == "Yes"]

In [214]:
df2 = basePrep(df2)

In [215]:
cols2 = ['ddist_bin','ddist_int', 'ddist_abs', 'avg_emp_pp', 'public_count']
X2 = (df2[cols2]).astype(float)
y2 = df2['pdynamic'].astype(float)
X2_constant = sm.add_constant(X2, prepend=False)

In [216]:
logit_model = GLM(y2, X2_constant, family=families.Binomial())
logit_results = logit_model.fit()
print(logit_results.summary())



                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                   66
Model:                            GLM   Df Residuals:                       60
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -33.305
Date:                Tue, 14 Jun 2022   Deviance:                       66.611
Time:                        14:41:14   Pearson chi2:                     61.8
No. Iterations:                     5   Pseudo R-squ. (CS):             0.1741
Covariance Type:            nonrobust                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
ddist_bin       -0.0651      1.194     -0.055   

## Filter for alliance SIC

In [217]:
df3 = pd.read_csv("./data/JV_data_dist.csv")

In [218]:
#Select alliances that at least part of their goal was manufacturing of cars
df3 = df3[df3['sic'].str.contains('3711') == True]


In [219]:
df3 = basePrep(df3)

In [220]:
cols3 = ['ddist_bin','ddist_int', 'ddist_abs', 'avg_emp_pp', 'public_count']
X3 = (df3[cols3]).astype(float)
y3 = df3['pdynamic'].astype(float)
X3_constant = sm.add_constant(X3, prepend=False)

In [221]:
logit_model = GLM(y3, X3_constant, family=families.Binomial())
logit_results = logit_model.fit()
print(logit_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  311
Model:                            GLM   Df Residuals:                      305
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -189.11
Date:                Tue, 14 Jun 2022   Deviance:                       378.23
Time:                        14:41:15   Pearson chi2:                     331.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1480
Covariance Type:            nonrobust                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
ddist_bin       -0.3361      0.506     -0.664   